In [4]:
## Can also get the price chart by pokemon number, or url for past prices. Look intro trading strategies; also, work on real-time offering feed vs history. 

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import re

# Customize number of pages to scrape
num_pages = 1
base_url = 'https://www.tcgplayer.com/search/pokemon/product?productLineName=pokemon&view=grid&Condition=Near+Mint|Lightly+Played&page={}&Language=English'

# Setup headless Chrome options
options = Options()
options.add_argument("--headless=new")
prefs = {"profile.default_content_setting_values": {"images": 2, "stylesheets": 2}}
options.add_experimental_option("prefs", prefs)

def scrape_page(page_num):
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    url = base_url.format(page_num)
    driver.get(url)

    try:
        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-card__product")))
    except:
        driver.quit()
        return []

    products = driver.find_elements(By.CLASS_NAME, "product-card__product")
    results = []

    for product in products:
        try:
            # Grab the link directly from the <a> tag
            link_elem = product.find_element(By.XPATH, "./ancestor::a")
            product_link = link_elem.get_attribute("href")
        except Exception:
            product_link = None

        try:
            img_elem = product.find_element(By.TAG_NAME, "img")
            name = img_elem.get_attribute("alt").strip()
        except Exception:
            name = None

        try:
            set_name = product.find_element(By.CLASS_NAME, "product-card__set-name__variant").text.strip()
        except Exception:
            set_name = None

        try:
            mktprice_text = product.find_element(By.CLASS_NAME, "product-card__market-price--value").text.strip()
            mktprice_match = re.search(r"\$([\d,.]+)", mktprice_text)
            mktprice = float(mktprice_match.group(1).replace(",", "")) if mktprice_match else None
        except Exception:
            mktprice = None

        try:
            listings_span = product.find_element(By.CLASS_NAME, "inventory__listing-count").text.strip()
            listings_match = re.search(r"(\d+)\s+listings", listings_span)
            listings = int(listings_match.group(1)) if listings_match else None
        except Exception:
            listings = None

        results.append({
            "name": name,
            "link": product_link,
            "set": set_name,
            "mktprice": mktprice,
            "listings": listings
        })

    driver.quit()
    return results

# Run across multiple pages
all_results = []
with ThreadPoolExecutor(max_workers=6) as executor:
    futures = [executor.submit(scrape_page, p) for p in range(1, num_pages + 1)]
    for future in as_completed(futures):
        all_results.extend(future.result())

# Convert to DataFrame
pricedf = pd.DataFrame(all_results)
print(pricedf)


                                             name  \
0        Code Card - Destined Rivals Booster Pack   
1            Code Card - White Flare Booster Pack   
2             Code Card - Black Bolt Booster Pack   
3   Code Card - Prismatic Evolutions Booster Pack   
4                                           Hilda   
5                                   Victini - 208   
6                                         Pikachu   
7                                 Arven - 166/198   
8                                     Air Balloon   
9                                  Iono - 080/091   
10                                   Prism Energy   
11                                Night Stretcher   
12                                Luminous Energy   
13                                   Brave Bangle   
14                        Black Bolt Booster Pack   
15                             Buddy-Buddy Poffin   
16                                         Kyogre   
17                       White Flare Booster P

In [6]:
import pandas as pd 
pd.set_option('display.max_colwidth', None)
pricedf

,name,link,set,mktprice,listings
0,Code Card - Destined Rivals Booster Pack,https://www.tcgplayer.com/product/633169/pokemon-sv10-destined-rivals-code-card-destined-rivals-booster-pack?Condition=Near+Mint|Lightly+Played&Language=English&page=1,SV10: Destined Rivals,0.05,223.0
1,Code Card - White Flare Booster Pack,https://www.tcgplayer.com/product/646130/pokemon-sv-white-flare-code-card-white-flare-booster-pack?Condition=Near+Mint|Lightly+Played&Language=English&page=1,SV: White Flare,0.08,97.0
2,Code Card - Black Bolt Booster Pack,https://www.tcgplayer.com/product/646128/pokemon-sv-black-bolt-code-card-black-bolt-booster-pack?Condition=Near+Mint|Lightly+Played&Language=English&page=1,SV: Black Bolt,0.05,99.0
3,Code Card - Prismatic Evolutions Booster Pack,https://www.tcgplayer.com/product/614046/pokemon-sv-prismatic-evolutions-code-card-prismatic-evolutions-booster-pack?Condition=Near+Mint|Lightly+Played&Language=English&page=1,SV: Prismatic Evolutions,0.04,227.0
4,Hilda,https://www.tcgplayer.com/product/642200/pokemon-sv-white-flare-hilda?Condition=Near+Mint|Lightly+Played&Language=English&page=1,SV: White Flare,3.50,158.0
5,Victini - 208,https://www.tcgplayer.com/product/646169/pokemon-sv-scarlet-and-violet-promo-cards-victini-208?Condition=Near+Mint|Lightly+Played&Language=English&page=1,SV: Scarlet & Violet Promo Cards,6.99,168.0
6,Pikachu,https://www.tcgplayer.com/product/250303/pokemon-celebrations-pikachu?Condition=Near+Mint|Lightly+Played&Language=English&page=1,Celebrations,7.24,265.0
7,Arven - 166/198,https://www.tcgplayer.com/product/488071/pokemon-sv01-scarlet-and-violet-base-set-arven-166-198?Condition=Near+Mint|Lightly+Played&Language=English&page=1,SV01: Scarlet & Violet Base Set,2.38,549.0
8,Air Balloon,https://www.tcgplayer.com/product/642531/pokemon-sv-black-bolt-air-balloon?Condition=Near+Mint|Lightly+Played&Language=English&page=1,SV: Black Bolt,1.53,257.0
9,Iono - 080/091,https://www.tcgplayer.com/product/534442/pokemon-sv-paldean-fates-iono-080-091?Condition=Near+Mint|Lightly+Played&Language=English&page=1,SV: Paldean Fates,0.35,912.0


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

options = Options()
# Keep browser visible but optimize resource use
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1280,800")

prefs = {"profile.default_content_setting_values": {"images": 2, "stylesheets": 2}}
options.add_experimental_option("prefs", prefs)

def scrape_product_page(url):
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 50)
    driver.get(url)
    try:
        table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table[role='region'][aria-live='polite']")))
        html_content = table.get_attribute("outerHTML")
    except Exception as e:
        print(f"Table not found or failed to load for {url}: {e}")
        html_content = None
    driver.quit()
    return url, html_content

# Example: assume pricedf has multiple rows with "link" column
urls = pricedf["link"].dropna().tolist()

# Limit number of concurrent threads to reasonable count (e.g., 4 or 5) to avoid overload
max_workers = 1

results = []
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    future_to_url = {executor.submit(scrape_product_page, url): url for url in urls}
    for future in as_completed(future_to_url):
        url, html = future.result()
        results.append({"url": url, "html": html})

# Print or further process results
for res in results:
    if res["html"]:
        print(f"Successfully scraped: {res['url']}")
    else:
        print(f"Failed to scrape: {res['url']}")


In [ ]:
from bs4 import BeautifulSoup
# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# Locate the table containing historical price data
price_table = soup.find("table", {"role": "region", "aria-live": "polite"})

# Extract rows from the table
rows = price_table.find_all("tr")[1:]  # Skip header row

# Parse each row into structured data
price_history = []
for row in rows:
    cells = row.find_all("td")
    if len(cells) >= 3:
        date_range = cells[0].get_text(strip=True)
        normal_price = cells[1].get_text(strip=True)
        alt_price = cells[2].get_text(strip=True)
        price_history.append({
            "Date Range": date_range,
            "Normal Price": normal_price,
            "Volume": alt_price 
        })

# Convert to DataFrame for analysis or export
import pandas as pd
history_df = pd.DataFrame(price_history)

# Display the result
print(history_df)


      Date Range Normal Price     Volume
0   5/17 to 5/19        $0.00      $0.00
1   5/20 to 5/22        $0.00      $0.00
2   5/23 to 5/25        $0.00     $67.00
3   5/26 to 5/28        $0.10  $5,246.00
4   5/29 to 5/31        $0.10  $4,695.00
5     6/1 to 6/3        $0.10  $3,681.00
6     6/4 to 6/6        $0.09  $1,361.00
7     6/7 to 6/9        $0.08    $813.00
8   6/10 to 6/12        $0.07    $750.00
9   6/13 to 6/15        $0.07  $3,208.00
10  6/16 to 6/18        $0.06  $1,818.00
11  6/19 to 6/21        $0.06  $1,209.00
12  6/22 to 6/24        $0.05  $1,459.00
13  6/25 to 6/27        $0.05  $1,471.00
14  6/28 to 6/30        $0.06  $1,149.00
15    7/1 to 7/3        $0.07  $1,018.00
16    7/4 to 7/6        $0.07    $835.00
17    7/7 to 7/9        $0.07  $1,714.00
18  7/10 to 7/12        $0.07    $879.00
19  7/13 to 7/15        $0.05  $1,323.00
20  7/16 to 7/18        $0.06    $923.00
21  7/19 to 7/21        $0.05  $1,776.00
22  7/22 to 7/24        $0.04  $1,363.00
23  7/25 to 7/27